# Merge Questionnaire Data

In [ ]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from fau_colors import cmaps

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

palette

In [ ]:
deploy_type = "local"

In [ ]:
config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])
base_path

In [ ]:
quest_path = base_path.joinpath("questionnaires")
quest_path_in = quest_path.joinpath("merged")
quest_path_out = quest_path.joinpath("merged_total")

bp.utils.file_handling.mkdirs([quest_path_in, quest_path_out])

In [ ]:
code_mapping = pd.read_csv(base_path.joinpath("extras/code_to_number_mapping.csv"))
code_mapping = code_mapping.set_index("Code")
code_mapping.head()

## Load Questionnaire Data and Map Code to VP-ID

In [ ]:
quest_data = pd.read_excel(quest_path_in.joinpath("questionnaire_data.xlsx"))
quest_data = quest_data.set_index("Code")
quest_data = quest_data.join(code_mapping).set_index("subject")
quest_data = quest_data.sort_index()
quest_data.head()

## Add Condition Order

In [ ]:
condition_order = pd.read_csv(base_path.joinpath("extras/condition_order.csv"))
condition_order = condition_order.set_index("subject")["condition_order"]
condition_order.head()

## Add PASA

In [ ]:
pasa_data = pd.read_excel(quest_path_in.joinpath("pasa.xlsx"))
pasa_data = pasa_data.set_index(["subject", "condition"])
pasa_data = pasa_data.join(condition_order).set_index("condition_order", append=True)
pasa_data = pasa_data.assign(**{"day": pasa_data.index.map("{0[1]}_{0[2]}".format)})
pasa_data = pasa_data.replace(
    {"tsst_tsst_first": "T1", "ftsst_tsst_first": "T2", "ftsst_ftsst_first": "T1", "tsst_ftsst_first": "T2"}
)
pasa_data = pasa_data.set_index("day", append=True)
pasa_data = pasa_data.droplevel(["condition", "condition_order"]).unstack("day")
pasa_data.columns = ["_".join(col) for col in pasa_data.columns]
pasa_data.head()

quest_data_out = quest_data.join(pasa_data)
quest_data_out.head()

## Add Condition Order

In [ ]:
quest_data_out.insert(0, "condition_order", condition_order)
quest_data_out.head()

## Add Handedness

In [ ]:
handedness = pd.read_excel(quest_path_in.joinpath("handedness.xlsx"))
handedness = handedness.set_index("subject")["handedness"]

quest_data_out.insert(1, "handedness", handedness)
quest_data_out.head()

## Export

In [ ]:
quest_data_out.to_excel(quest_path_out.joinpath("questionnaire_data.xlsx"))